# Week 6 Exercises

## Exercise 1: Neural Network Design (Don't spend too much time)
In this exercise your task is to design neural networks by hand that compute simple functions.
For the nonlinear transform you can mix them any way you like but you can only use, identity, sign, relu and sigmoid transforms in the neurons.
You can make the networks as wide and deep as you would like but small networks are sufficient.
* Make a network that computes $c \cdot x$ for any constant c
* Make a network that computes xor of inputs $x_1$ and $x_2$, when $x_1, x_2 \in \{0, 1\}$ ($x_1$ xor $x_2$ is $0$ when $x_1$ and $x_2$ are equal, and it is $1$ otherwise).
* Make a network that computes max($x_1$,$x_2$)
* Make a network that computes $x^2$ - for any integer x in the set {2,3,4,5} <b>(Might be hard so skip if waste too much time)</b>


- **Hint 1: It may be easier to find an easy mathematical expression that solves the problem and then to make a network that implements that**
- **Hint 2: It suffices to use just relu and identity activations**.

## SOLUTION MATH HERE
A single neuron. Weights c. Activation is identity.

In first layer: One neuron with weights (1,1) and bias -1, taking ReLU. Another neuron with weights (-1,-1) bias +1, taking ReLU. Output layer one neuron with weight (-1,-1) and bias 1, identity.

In first layer: One neuron with weights (1,-1), taking ReLU. Another neuron with weights (-1,1) taking ReLU. And a neuron computing the sum of the two inputs using weights (1,1), identity. Output layer has weights (1/2,1/2,1/2) and identity. Observe: Sum of first two neurons in first layer is max(x_1,x_2)-min(x_1,x_2). Adding x_1+x_2 cancels out the smallest and leaves twice the max. Dividing it all by 2 gives the desired result.

We create a small network testing whether x = y. This is done by having two neurons, one having weight -1 and bias +y and another having weight 1 and bias -y. Both have ReLU. If x=y then both are 0. Otherwise, one is 0 and one is >= 1. Now create a new neuron taking the two previous as inputs with weight -1000 on both and bias y^2, taking ReLU. This becomes y^2 iff x=y, and otherwise it becomes 0. Do this for every y in 2,3,4,5 and create a neuron with weight 1,1,1,1 summing these and using identity.
## END SOLUTION

## Ex 2: Neural Net Forward Pass - Vectorized
Implement the score (least squares error), and predict function for a neural net class for regression.
The neural net has one hidden layer with  $\textrm{relu}(x) = \max(0, x)$ activation and one output neuron with identity activation.

For the prediction method you must write an algorithm that takes as input a batch of data and computes the output of the neural net on each input point given in the batch.

The data batch is given as an $n \times d$ matrix $X$, where each row is a data point.


A neural net as considered here requires two sets of weights and biases
* The weights that map the input data to the input to hidden units. Call that W_1. The bias weights for this we name $b_1$.

* The weights that map the output of the hidden units to the output. Call that W_2. The bias weights for this we name $b_2$.

We organize the weighs in matrices $(W_1, W_2)$ and vectors $(b_1,b_2)$ as follows (consistent with slides):
* The $i'th$ column of $W_1$ are the weights we multiply with the input data to get the input to hidden node $i$. The shape of the $W_1$ matrix is $d \times h$.
* The bias $b_1$ is a vector of size h, the i'th entry is the bias to hidden neuron $i$.
* The $i'th$ column of $W_2$ are the weights we multiply with the hidden layer activations to get the input to the i'th output node. $W_2$ is a $h \times \textrm{output_size}$ matrix ($h \times 1$ matrix in our case)
* The bias $b_2$ is a vector of size output_size 

**Task:** In the cell below (partially) complete the neural net class
- Implement the predict function of the neural net
- Implement the score function (least squares $\frac{1}{n} \sum_i (\textrm{nn}(x_i) - y_i)^2$)

Tests:
- We have a simple test case with random weights. The actual error here is random since we just set random weights.
- The second test case uses the weight of a pretrained network for house pricing. Here you should get a score around 0.32 (remove comment to run it)


In [ ]:
%matplotlib inline
import numpy as np

class NN():
    
    def __init__(self, input_dim, hidden_size):
        output_size = 1
        self.W1 = np.random.rand(input_dim, hidden_size)
        self.b1 = np.random.rand(1, hidden_size)
        self.W2 = np.random.rand(hidden_size, output_size)
        self.b2 = np.random.rand(1, output_size)
        print('Neural net initialized with random values')
        
    def predict(self, X):    
        """ Evaluate the network on given data batch 
        
        np.maximum may come in handy
        
        Args:
        X: np.array shape (n, d)  Each row is a data point
        
        Output:
        pred: np.array shape (n, 1) output of network on each input point
        """
        # compute the following values
        pred = None # the output of neural net n x 1
    
        ### YOUR CODE HERE
        hiddenIn = X @ self.W1 + (np.ones((X.shape[0],1)) @ self.b1) #n x h
        nonLinear = np.maximum(hiddenIn,0) #n x h
        pred = nonLinear @ self.W2 + (np.ones((X.shape[0],1)) @ self.b2) #n x o
        ### END CODE
        return pred
    
    
    def score(self, X, y):
        """ Compute least squares loss (1/n sum (nn(x_i) - y_i)^2)
        
          X: np.array shape (n, d) - Data
          y: np.array shape (n, 1) - Targets

        """
        score = None
        ### YOUR CODE HERE
        pred = self.predict(X)
        score = np.mean((pred-y)**2)
        ### END CODE
        return score
        
# random data test
def simple_test():
    input_dim = 3
    hidden_size = 8
    X = np.random.rand(10, input_dim)
    y = np.random.rand(10, 1)
    my_net = NN(input_dim=input_dim, hidden_size=hidden_size)

    nn_out = my_net.predict(X)
    print('shape of nn_out', nn_out.shape) # should be n x 1
    print('least squares error: ', my_net.score(X, y))
    
# actual data test
def housing_test():
    from sklearn.preprocessing import StandardScaler
    from  sklearn.datasets import fetch_california_housing
    import os, ssl
    if (not os.environ.get('PYTHONHTTPSVERIFY', '') and
        getattr(ssl, '_create_unverified_context', None)): 
        ssl._create_default_https_context = ssl._create_unverified_context
    
    rdata = fetch_california_housing()
    s = StandardScaler()
    Xr = rdata.data
    yr = rdata.target   
    print('data size:', len(yr), 'num features:', Xr.shape[1])
    s.fit(Xr)
    X_scaled = s.transform(Xr)
    house_net = NN(input_dim=Xr.shape[1], hidden_size=8)
    weights = np.load('good_weights.npz')
    house_net.W1 = weights['W1']
    house_net.W2 = weights['W2']
    house_net.b1 = weights['b1'].reshape(1, -1)
    house_net.b2 = weights['b2'].reshape(1, -1)
    print('hidden layer size:', house_net.W1.shape[1])
    lsq = house_net.score(X_scaled, yr.reshape(-1, 1))
    pred = house_net.predict(X_scaled)
    print('mean house price least squares error:', lsq)
    #print('5 house prediction:\nestimated price , true price')
    #print(np.c_[house_net.predict(X_scaled[0:5, :]), yr[0:5]])

simple_test()
housing_test()

## Ex: 3 Install and test Pytorch
In this exercise we will use the deep learning frame work pytorch. So install it (you might be able to install it by running the code below). The package name is torch.

In this exercise, your task is to use automatic differentation to implement gradient descent for Linear Regression (rather than computing the analytic exact solution).

First we need to understand that to represent data we must use torch tensors. Tensors are very much like numpy arrays just with some extra functionality.
The thing we will consider is the backward function that computes gradients of whatever computation you have made using torch tensors.

To see how this works, let's see an example.
Let's evaluate the gradient of the sigmoid function without actually knowing the formula. All you need to know is how to compute the function using standard functions i.e. $s(z) = 1/(1+e^{-z})$.



In [ ]:
!pip install torch

In [ ]:
import torch
z = torch.zeros(1, requires_grad=True)
sz = 1.0 / (1+ torch.exp(-z))
sz.backward() # compute gradient of sz relative to z in this case
print('Gradient of f(z)=1/(1 + e^{-z}) at z = 0', z.grad)


## Ex 3 Continued: Gradient Descent with pytorch
In deep learning frameworks all we usually need to do, is to show how to compute the cost in a given point and the system then automatically computes the gradient in that data point for you. In this exercise, we will try and see if we can use this functionality to do gradient descent.


**Setup:**
We create a data set $D$ that will consist of $n=100$ data points in 2D $(x_1,x_2)$ i.e. two features $x_1, x_2$. 
For some reason, the data feature vectors of $x_1$ and $x_2$ (the length $n$ vectors with all data values for the corresponding feature) in our experiment are orthogonal and $x_1$ has unit norm while $x_2$ has norm $a$.

We generate a target vector 
$$
y = x_1+x_2
$$ 
which is also a vector of length $n$ i.e. the data we are trying to fit comes from *(a very simple)* linear model.

Remember for linear regression, the in sample error is:
$$
\textrm{E}_\textrm{in}(w) = \frac{1}{n} \sum_{i=1}^n (w^\intercal x_i - y_i)^2 = \frac{1}{n} \|Xw -y\|^2
$$


We have written the code to generate data and the surrounding Gradient Descent for loop, all you need is to write the code for computing the cost (ein).
You can only use commands from torch here (no numpy), but you can use standard operators like $+,-,*,/,**$ on torch tensors that work like their numpy equivalent and torch.sum may be very handy
**Complete the gradient descent code below by computing cost using standard operations and torch commands only**

The gradient descent will start the search at $w=(42, 2)$ for some reason. We have also sat an almost arbitrary learning rate. You can change both if you like.

**For the $a$ specified, the lecturer gets an epoch 41 cost of approximately 0.25 and a w of approximately [5.75, 0.25]**

In [ ]:
import numpy as np 
import torch # the torch tensor library

## CREATE SOME DATA
n = 100
x1 = np.random.rand(n)
x2 = np.random.rand(n)
## Grahm schmidt process - ignore
x1 = x1/np.linalg.norm(x1)
x2 = x2/np.linalg.norm(x2)
x2 = x2 - np.dot(x1, x2) * x1 #
x2 = x2/np.linalg.norm(x2)

## CREATE THE DATA MATRIX
a = 4.0
D = np.c_[x1, a*x2]
# CREATE TARGET FUNCTION VECTOR
y = x1 + x2

# MAKE TORCH VARIABLES TO USE
X = torch.from_numpy(D).double()
ty = torch.from_numpy(y).double()
ni = torch.tensor(1./n, dtype=torch.double)

def torch_gd():
    w = torch.tensor([42.0, 2.0], dtype=torch.double)
    lr = torch.tensor(10.0/a, dtype=torch.double)
    epochs = 42
    cost_hist = []
    for i in range(epochs):
        w.requires_grad_() # say i want gradient relative to w in upcoming computation
        cost = None
        ### YOUR CODE HERE - Compute Ein for linear regression as function of w and store in cost 1 - 4 lines
        cost =  torch.mean((X @ w - ty)**2)
        ### END CODE
        cost_hist.append(cost)
        print('epoch {0}: Cost {1}'.format(i, cost))
        cost.backward() # compute gradient cost as function of w
        w = w - lr * w.grad # update w
        w.detach_() # removes w from current computation graph
    print('best w found', w)
torch_gd()

## Exercise 4: Backpropagation on small example, manually

Consider the simple neural net 
$$
\textrm{nn}(x_1,x_2) = w_3 \cdot \textrm{relu}(w_1 \cdot x_1 + w_2 \cdot x_2)
$$
where $\textrm{relu}(x) = \max(x,0)$ and $w_1, w_2, w_3$ are the weights of the network

We only have one input point $x=(x_1,x_2) = (3.0, 1.0)$ with target value $y=9.0$

Consider the least squares error 
$$
e=(y-\textrm{nn}(x))^2
$$

We need to minimize the error over $w_1, w_2, w_3$ and will do that using the gradients of $e$ relative to $w_1, w_2, w_3$.
Initialize $w_1=1,w_2=2, w_3=1$

Draw the computational graph for $e$ and run the forward pass to compute the output of the neural network on $x$ and the least squares error, and then run the backwards pass to compute the partial derivative of the loss relative to $w_1,w_2, w_3$ on the fixed input $x,y$ with the fixed values for $w_1,w_2,w_3$.

Write the python code that performs the forward and backwards pass below and evaluate the cost and the gradient
using notation similar to
http://cs231n.github.io/optimization-2/

Print intermediate steps in both the forward and the backward pass.

In [ ]:
print('Do the python forward and backward pass here')
x1 = 3.0
x2 = 1.0 
y = 9.
w1 = 1.0
w2 = 2.0
w3 = 1.0
### YOUR CODE HERE
# Long Forward Pass
z1 = w1 * x1
z2 = w2 * x2
hin = z1 + z2
hout = max(hin, 0)
pred = w3 * hout
diff = pred - y
e = diff**2
print('z1', z1)
print('z2', z2)
print('hin', hin)
print('hout', hout)
print('pred', pred)
print('diff', diff)
print('ls error', e)

# backwards pass
de_diff = 2 * diff
ddiff_pred = 1
de_pred = de_diff * ddiff_pred
dpred_w3 = hout
de_w3 = de_pred * dpred_w3 # dout_w3 = rs
dpred_hout = w3
de_hout = de_pred * dpred_hout
dhout_hin = hin > 0
de_hin = de_hout * dhout_hin
dhin_z1 = 1
dhin_z2 = 1
de_z1 = de_hin * dhin_z1
de_z2 = de_hin * dhin_z2
dz1_w1 = x1
dz2_w2 = x2
print('dz', dz1_w1, dz2_w2)
de_w1 = de_z1 * dz1_w1
de_w2 = de_z2 * dz2_w2
print('d_diff', de_diff)
print('d_pred', de_diff)
print('d_hout', de_diff)
print('d_hin', de_diff)
print('d_w1:', de_w1)
print('d_w2:', de_w2)
print('d_w3:', de_w3)
### END CODE



## Ex 5: Automated Backpropagation using pytorch
In this exercise we will check our results from the previous exercise using pytorch.

For this we only need to code the forward pass and let automatic differentation take care of the rest!

**Task:** Write the forward pass in the cell below and use automatic differentation to test your answer from above.

Use x.retain_grad() to keep the gradient of any intermediate computation used in the forward pass to compare with above



In [ ]:
import torch
from torchviz import make_dot # install this package 

x1 = torch.tensor([[3.]])
x2 = torch.tensor([[1.]])
y = torch.tensor([9.])
W1 = torch.tensor([[1.]], requires_grad=True)
W2 = torch.tensor([[2.]], requires_grad=True)
W3 = torch.tensor([[1.]], requires_grad=True)
### YOUR CODE HERE - The clamp function may be usefull
hin = x1 @ W1 + x2 @ W2
hout = hin.clamp(min=0)
pred = hout * W3
diff = pred - y
loss = diff**2
print('forward pass')
print('least squares error', loss)
hin.retain_grad()
hout.retain_grad()
pred.retain_grad()
diff.retain_grad()
loss.backward()
print('d_diff', diff.grad)
print('d_pred', pred.grad)
print('d_hout', hout.grad)
print('d_hin', hin.grad)
### END CODE
# print the graph - change naming appropriately
print('d_w1', W1.grad)
print('d_w2', W2.grad)
print('d_w3', W3.grad)

## Ex. 6: Matrix of Derivatives for applying function elementwise to vector 
In this exercise we consider the matrix of derivatives when mapping a vector of size $n$ to a vector of size $n$ by applying a function $f$ to each entry in the input.

Let $f$ be a differentiable function from $\mathbb{R} \rightarrow \mathbb{R}$.

Let $F: \mathbb{R}^d \rightarrow \mathbb{R^d}$ defined as $F(v) = \left[f(v_1), \dots, f(v_d)\right]$

- Show that the Matrix of Derivatives $D_f$ is the $d \times d$ diagonal matrix where the $i$'th diagonal entry is $f'(v_i)$.
- If $f(x) = 1/(1+e^{-x})$ i.e. the logistic sigmoid function then how does the Diagonal matrix look?
- If $f(x) = \max(0, x)$ i.e. relu how does the Diagonal Matrix look?
- Consider a neural net backpropagation step and let hout be the output of applying F to the vector hin and assume we have computed the vector $\frac{\partial L}{\partial \textrm{hout}}$ (of shape $1 \times h$)
    and wish to compute  $\frac{\partial L}{\partial \textrm{hin}}$ (also $1\times h)$. Write the python code that achieves this below.
    

**hint:** There may be a more efficient way than actually creating the diagonal matrix and multiplying on the backpropagated derivative but you do not have to optimize.

## SOLUTION MATH HERE
 The matrix of derivatives is a $d \times d$ matrix
 
 The derivative of $f_i = f(v_i)$ after $v_j$ is zero if $i \neq j$. So we get a diagonal matrix.
 
 If i==j then we need the derivative of $f(v_i)$ as a function of v_i which is $f'(v_i)$.
 
 For sigmoid we get $D_{i,i} = f(v_i)(1-f(v_i)) = f(v_i)f(-v_i)$. 
 
 For relu we get $D_{i, i} = 1_{v_i > 0}$ (indicator variables)
## END SOLUTION

In [ ]:
import numpy as np
h_in = np.array([[-1, 2, 4]])
d_hout = np.array([[1,2,3]])
print('shapes:', h_in.shape, d_hout.shape)
def relu_grad(d_hout, hin):
    d_hin = None
    ### YOUR CODE HERE
    d_hin = d_hout.copy()
    d_hin[hin<0] = 0
    ### END CODE
    return d_hin

def sigmoid_grad(d_hout, hin):
    d_hin = None
    ### YOUR CODE HERE
    def sigmoid(x):
        return 1./(1+np.exp(-x))
    d_hin =  d_hout * (sigmoid(hin)*(1-sigmoid(hin))) # entrywise multiplication
    ### END CODE
    return d_hin

print('d_hin relu:', relu_grad(d_hout, h_in))
# should be [0, 2, 3]
print('d_hin sigmoid:', sigmoid_grad(d_hout, h_in))
# should be ~ [0.196..., 0.209..., 0.052...]


# Ex 7: Neural Nets by automatic differentation in pytorch
In this exercise we will use pytorch to implement a pair of basic basic neural nets and train them using just automatic differentation and the optim module from pytorch that implements different gradient based optimization methods.

**The task is:**

Using pytorch, implement Linear Regression with weight decay
(named Ridge Regression) using Gradient Descent as the Learning Algorithm. Next, implement a one hidden layer neural net with relu activation for regression (identity activation on the output neuron) with the same cost function.

Let $w, b$ be the parameters of the function $f$ we are trying to learn, then least squares with weight decay cost for weight decay parameter $\lambda$ is
$$
\frac{1}{n} \sum_{i=1}^n (f(x)-y)^2 + \lambda \sum_{i=1}^d w_i^2
$$
Note we do not penalize the bias weights.

1. Implement Linear Regression with Gradient Descent and test on the California housing data set for house prices (see cell below).

   Linear regression is simply a neural net with no hidden layer and one output neuron. We have implemented the fit methods to show you how to use the optim module.
   That means all you need to do is implement the cost method.
2. Implement a standard one hidden layer neural net for regression. That means identity output activation and least squares error with weight decay. The formula for weight decay is the same, i.e. you need to add the squared weight to the error for all weights in all layers. 
For nonlinearity use relu. See cell two below.

   In this exercise you need to complete the following two methods (hint: fit is similar to linear regression fit)
   - cost - compute the regularized least squares cost of the network on data and return the pytorch tensor of that
   - fit - train for 1000 steps of gradient descent using optim package - find a good learning rate your self
   - try to run for only 100 steps (or fewer) of gradient descent. How well does it perform (in-sample)?

We test your implementation on a standard regression data set for house prices and compare to the sklearn build in Ridge Regression. 
We only consider in sample error, which is of course not what we care about in the real world!.

Your Linear Regression implementation should get close to the sklearn Ridge Regression implementation that we have included

For the Neural Net you should do better with the right learning rate (better in-sample)

In [ ]:
import torch
from torch import optim

from sklearn.datasets import fetch_california_housing

import os, ssl
if (not os.environ.get('PYTHONHTTPSVERIFY', '') and
    getattr(ssl, '_create_unverified_context', None)): 
    ssl._create_default_https_context = ssl._create_unverified_context

from sklearn import linear_model

print('*'*5, 'Load and Prepare Data', '*'*5)
dataset = fetch_california_housing()
# print('dataset', dataset)
X, y = dataset.data, dataset.target
X = (X - X.mean(axis=0))/(X.std(axis=0))
#print('data stats', X.shape, X.mean(axis=0), X.std(axis=0))
ridge=linear_model.Ridge(alpha=0.1, fit_intercept=True)
ridge.fit(X, y)
# print(ridge.coef_, ridge.intercept_)
print('\n', '*'*5, 'Test Sklearn Ridge Regression for Comparison', '*'*5)
print('Ridge Regression Score:', ((ridge.predict(X)-y)**2).mean())

print('\n', '*'*5, 'Make data to torch tensors', '*'*5)
tX = torch.from_numpy(X).float()
ty = torch.from_numpy(y).float().view(-1, 1)


class LR():
    
    def __init__(self):
        pass
        
    def cost(self, X, y, w, b, c=0):
        """ Compute Regularized Least Squares Loss
        
          X: torch.tensor shape (n, d) - Data
          y: torch.tensor shape (n, 1) - Targets
          w: torch.tensor shape (d, 1) - weights
          b: torch.tensor shape (1, 1) - bias weight
          c: scalar, weight decay parameter 
          
          returns (regularized) cost tensor        
        """
        loss = None
        ### YOUR CODE HERE
        pred = X @ w + b 
        loss = torch.mean((pred-y)**2) 
        reg_loss = torch.sum(w**2)
        ### END CODE
        return loss + c * reg_loss
    
    def fit(self, X, y, c=0):
        """ GD Learning Algorithm for Ridge Regression with pytorch
        
        Args:
         X: torch.tensor shape (n, d)
         y: torch.tensor shape (n, 1)
         c: ridge regression weight decay parameter (lambda)
        """
        w = torch.zeros(X.shape[1], 1, requires_grad=True)
        b = torch.zeros(1, 1, requires_grad=True)
        sgd = optim.SGD(params={w, b}, lr=0.1)
        for i in range(100):
            sgd.zero_grad()
            loss = self.cost(X, y, w, b, c=c)
            if i % 10 == 0:
                print('epoch:', i, 'least squares (regularized loss)', loss.item())
            loss.backward()
            sgd.step()
        self.w = w.clone()
        self.b = b.clone()


    def score(self, X, y):
        """ Compute least squares cost for model 
        
        Args:
         X: torch.tensor shape (n, d)
         y: torch.tensor shape (n, 1)
         
        returns least squares score of model on data X with targets y
        """
        score = self.cost(X, y, self.w, self.b, c=0)
        return score

print('\n', '*'*5, 'Run Torch Linear Regression Gradient Descent', '*'*5)

tlr = LR()
tlr.fit(tX, ty, 0.1)
print('pytorch Linear Regression least squares score:', tlr.score(tX, ty).item())

In [ ]:
import torch
from torch import optim
from sklearn import linear_model

from sklearn.datasets import fetch_california_housing

import os, ssl
if (not os.environ.get('PYTHONHTTPSVERIFY', '') and
    getattr(ssl, '_create_unverified_context', None)): 
    ssl._create_default_https_context = ssl._create_unverified_context

dataset = fetch_california_housing()
X, y = dataset.data, dataset.target
X = (X - X.mean(axis=0))/(X.std(axis=0))
tX = torch.from_numpy(X).float()
ty = torch.from_numpy(y).float().view(-1, 1)

class NN():
    
    def __init__(self):
        pass

    def cost(self, X, y, W1, b1, W2, b2, c=0):
        """ Compute (Regularized) Least Squares Loss of neural net
        The clamp function may be usefull
        
          X: torch.tensor shape (n, d) - Data
          y: torch.tensor shape (n, 1) - Targets
          W1: torch.tensor shape (d, h) - weights
          b1: torch.tensor shape (1, h) - bias weight
          W2: torch.tensor shape (h, 1) - weights
          b2: torch.tensor shape (1, 1) - bias weight
          c: ridge regression weight decay parameter 
    
        returns (weight decay) cost tensor
        """
   
        loss = None
        ### YOUR CODE HERE
        hin = X @ W1 + b1
        hout = hin.clamp(min=0)
        pred = hout @ W2 + b2
        loss = torch.mean((pred-y)**2) 
        reg_loss = c * torch.sum(W1**2) + c * torch.sum(W2**2)
        loss = loss + reg_loss
        ### END CODE
        return loss
    
    def fit(self, X, y, hidden_size=32, c=0.01):   
        """ GD Learning Algorithm for Ridge Regression with pytorch
        
         Args:
         X: torch.tensor shape (n, d)
         y: torch.tensor shape (n, 1)
         hidden_size: int 
         c: float weight decay parameter (lambda)
        """
        input_dim = X.shape[1]        
        W1 = torch.randn(input_dim, hidden_size, requires_grad=True)
        b1 = torch.randn(1, hidden_size, requires_grad=True)
        W2 = torch.randn(hidden_size, 1, requires_grad=True)
        b2 = torch.randn(1, 1, requires_grad=True)
        ### YOUR CODE HERE
        sgd = optim.SGD(params={W1, W2, b1, b2}, lr=0.03)
        for i in range(1000):
            sgd.zero_grad()
            loss = self.cost(X, y, W1, b1, W2, b2, c=c)
            if i % 10 == 0:
                print('epoch:', i, 'nn least squares loss (regularized)', loss.item())
            loss.backward()
            sgd.step()
        ### END CODE
        self.W1 = W1
        self.W2 = W2
        self.b1 = b1
        self.b2 = b2
        
    def score(self, X, y):
        """ Compute least squares cost for model 
        
        Args:
         X: torch.tensor shape (n, d)
         y: torch.tensor shape (n, 1)
         
        returns least squares score of model on data X with targets y
        """
        score = self.cost(X, y, self.W1, self.b1, self.W2, self.b2, c=0)
        return score


net = NN()
net.fit(tX, ty, hidden_size=16, c=0.01)
print('pytorch Neural Net least squares score:', net.score(tX, ty).item())